For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: Andrew Lin, Praneeth Prathi, Zibing Zhang

Step 1: Train your own word embeddings
--------------------------------

**TODO**: ADD TO DESCRIPTIONS, general properties, etc.

<!-- (describe the provided dataset that you have chosen here) -->
The first dataset that we are using was provided by the instructors.
It is the complete works of Shakespeare.
It seems to have already gone through a large amount of normalization, as there are not punctuation marks, and everything is lowercased.
This poses some issue since we do not know sentence boundaries, but we will treat separate lines as separate sentences.

<!-- Describe what data set you have chosen to compare and contrast with the your chosen provided dataset. Make sure to describe where it comes from and it's general properties. -->

<!-- (describe your dataset here) -->
The second dataset that we are using is from the nltk gutenberg corpus.
We have chosen the King James version of the Bible.
The data is already split by whitespace and punctuation.

In [ ]:
# import your libraries here

import nltk
import re

In [ ]:
# setup datasets and pre-process into desired format

# Read the file and prepare the training data 
# so that it is in the following format

# data = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
# 			['this', 'is', 'the', 'second', 'sentence'],
# 			['yet', 'another', 'sentence'],
# 			['one', 'more', 'sentence'],
# 			['and', 'the', 'final', 'sentence']]

START_TOKEN = "<s>"
END_TOKEN = "</s>"

# shakespeare, given dataset
dataset_1 = []
with open("shakespeare_plays.txt") as f:
  dataset_1 = [[START_TOKEN] + line.split() + [END_TOKEN] for line in f.read().strip().split("\n")]
# print(dataset_1[0])  # first sentence

# king james version of bible, our datset
dataset_2 = []
nltk.download("gutenberg")
kjv = nltk.corpus.gutenberg.words("bible-kjv.txt")
# print(len(kjv))  # 1010654 words
sentence = [START_TOKEN]
for word in kjv:
  # lower case words
  word = word.lower()
  # period denotes end of sentence
  if word == ".":
    sentence.append(END_TOKEN)
    dataset_2.append(sentence)
    sentence = [START_TOKEN]
  # only allow words that contain only letters
  elif not re.match(r"^[a-z]+$", word):
    continue
  else:
    sentence.append(word)
print(dataset_2[0])  # first sentence

In [ ]:
# code to train word embeddings

from gensim.models import Word2Vec

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim.
# The default arguments are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
def train_embeddings(dataset, embeddings_size=EMBEDDINGS_SIZE, sg=1, window=5, min_count=1):
  return Word2Vec(sentences=dataset, vector_size=embeddings_size, sg=sg, window=window, min_count=min_count)


### a) Train embeddings on GIVEN dataset

In [ ]:
model_1 = train_embeddings(dataset_1)

In [ ]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
# print('Vocab size {}'.format(len(model.wv.vocab)))  # outdatted in gensim 4.0.0
print('Vocab size {}'.format(len(model_1.wv.index_to_key)))

In [ ]:
# You can save file in txt format, then load later if you wish.
# model_1.wv.save_word2vec_format('embeddings.txt', binary=False)

### b) Train embedding on YOUR dataset

In [ ]:
model_2 = train_embeddings(dataset_2)

In [ ]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
# print('Vocab size {}'.format(len(model.wv.vocab)))  # outdatted in gensim 4.0.0
print('Vocab size {}'.format(len(model_2.wv.index_to_key)))

In [ ]:
# You can save file in txt format, then load later if you wish.
# model_2.wv.save_word2vec_format('embeddings.txt', binary=False)

__What text-normalization and pre-processing did you do and why?__

For the given dataset, there was not any pre-processing that was done, because the dataset was already in a format that was very much well suited to modeling.
Everything was lower cased, and there was no punctuation.
This did pose some problem, as there was no clear delimiter for when a sentence would begin and end, but we decided to split the corpus into sentences on newlines.

The King James Version of the Bible required some text-normalization and pre-processing.
The corpus was already split into tokens, but we filtered some out and added some text-normalization.
To be overly cautious, we discarded all tokens that contained more than just alphabetic characters.
We also lower cased all the words.

Step 2: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

In [ ]:
def distance(wv1, wv2):
  d = 0
  for x1, x2 in zip(wv1, wv2):
    d += (x1 - x2) ** 2
  return d ** 0.5

# i think distance and angle, we probably only need one, since i'd imagine the vectors are somehow normalized to distance 1, unsure..., should check
def length(wv):
  return sum(e ** 2 for e in wv) ** 0.5
def angle(wv1, wv2):
  import math
  return math.acos(sum(e1 * e2 for e1, e2 in zip(wv1, wv2)) / (length(wv1) * length(wv2)))

# maybe compare common words between the two?
# for word in ["apple", "boat", "god", "jesus", "happy", "sad", "sword", "good", "well"]:
#   print(distance(model_1.wv[word], model_2.wv[word]))

print(distance(model_1.wv["kitchen"], model_2.wv["woman"]))
print(distance(model_1.wv["kitchen"], model_2.wv["life"]))

print(angle(model_1.wv["kitchen"], model_2.wv["woman"]))
print(angle(model_1.wv["kitchen"], model_2.wv["life"]))

##Write down your analysis:

Cite your sources:
-------------

Step 3: Feedforward Neural Language Model
--------------------------

### a) First, encode  your text into integers

In [ ]:
# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding

# The size of the ngram language model you want to train
# change as needed for your experiments
NGRAM = 3 

# Initializing a Tokenizer
# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(data)
# encoded = tokenizer.texts_to_sequences(data)


### b) Next, prepare your sequences from text

#### Fixed ngram based sequences 

In [ ]:
def generate_ngram_training_samples(ngram: list) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    pass


### c) Then, split the sequences into X and y and create a Data Generator

In [ ]:
# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]

In [ ]:
def read_embeddings():
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    pass



In [ ]:
def data_generator(X: list, y: list, num_sequences_per_batch: int) -> (list,list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    '''
    pass



In [ ]:
# Examples
# initialize data_generator
# num_sequences_per_batch = 128 # this is the batch size
# steps_per_epoch = len(sequences)//num_sequences_per_batch  # Number of batches per epoch
# train_generator = data_generator(X, y, num_sequences_per_batch)

# sample=next(train_generator) # this is how you get data out of generators
# sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
# sample[1].shape   # (batch_size, |V|) to_categorical

### d) Train your models

In [ ]:
# code to train a feedforward neural language model 
# on a set of given word embeddings
# make sure not to just copy + paste to train your two models

# Define the model architecture using Keras Sequential API



In [ ]:
# Start training the model
model.fit(x=train_generator, 
          steps_per_epoch=steps_per_epoch,
          epochs=1)

### e) Generate Sentences

In [ ]:
# generate a sequence from the model
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 seed: list, 
                 n_words: int):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
    Returns: string sentence
    '''
    pass

### f) Compare your generated sentences

Sources Cited
----------------------------
